In [10]:
import matplotlib
matplotlib.use('TkAgg')  # Set Matplotlib backend to TkAgg (for interactive plotting on some systems)

import numpy as np
import matplotlib.pyplot as plt  # Standard plotting
from matplotlib.animation import FuncAnimation  # Optional: for animations

from shapely.geometry import LineString, Point  # For geometric computations
from shapely.strtree import STRtree  # For spatial indexing of geometries
from collections import defaultdict

import networkx as nx # reconstruct network

import random  # For Python’s own RNG (not used yet)

plt.rc("text", usetex=False)  # Use standard text rendering (no LaTeX)
plt.rc("font", family = "serif")  # Set font to serif
plt.rc("figure", figsize=(10,8))  # Default figure size

%config InlineBackend.figure_format = 'retina'  # Jupyter setting for high-res inline plots (won’t work in script mode)

In [11]:
class Particle:
    def __init__(self, x, y, p_run, p_tumble, theta, speed, p_link, L_link_mean, L_link_std):
        self.position = np.array([x, y])  # Current position (in unit square)
        self.p_run = p_run  # Probability of running at each timestep
        self.p_tumble = p_tumble  # Probability of tumbling (direction change)
        self.theta = theta  # Current heading (angle in radians)
        self.speed = speed  # Speed per timestep
        self.p_link = p_link  # Probability of laying a link at each timestep
        self.L_link_mean = L_link_mean  # Mean link length
        self.L_link_std = L_link_std  # Std dev of link length

    def maybe_lay_link(self):
        if np.random.rand() < self.p_link:
            # Sample link length from Gaussian, fold negative to positive
            L_link = np.abs(np.random.normal(self.L_link_mean, self.L_link_std))
            return True, L_link
        return False, 0.0

    def run(self, dt):
        if np.random.rand() < self.p_run:
            displacement = self.speed * dt # how far does the particle run?
            dx = displacement * np.cos(self.theta) # change in x coordinate (w/ orientation)
            dy = displacement * np.sin(self.theta) # change in y coordinate (w/ orientation)
            self.position = (self.position + np.array([dx, dy])) % 1.0  # Enforce periodic boundaries

    def tumble(self):
        if np.random.rand() < self.p_tumble:
            self.theta = np.random.uniform(-np.pi, np.pi)  # Randomly sample new direction

edges = []  # List of edges as ((x1, y1), (x2, y2)) tuples
nodes = []  # List of node positions (redundant in current logic)

def add_link(particle, L_link):
    # Compute the endpoints of the link centered at particle's position and aligned with heading
    dx = 0.5 * L_link * np.cos(particle.theta)
    dy = 0.5 * L_link * np.sin(particle.theta)
    start = (particle.position - np.array([dx, dy])) % 1.0  # Periodic wrap
    end = (particle.position + np.array([dx, dy])) % 1.0 # Periodic wrap
    edges.append((start, end))  # Store edge
    nodes.append(particle.position.copy())  # Currently adds a node at every deposition (may not be desired)

def simulate(N, steps, dt, particle_params):
    global nodes, edges
    nodes, edges = [], []  # Clear previous run data

    # Create N particles with randomized initial positions and angles
    particles = [
        Particle(
            x=np.random.rand(), y=np.random.rand(),  # Random position
            theta=np.random.uniform(-np.pi, np.pi),  # Random direction
            **particle_params  # Fill in other parameters
        )
        for _ in range(N)
    ]

    for step in range(steps):
        for p in particles:
            laid_link, L_link = p.maybe_lay_link()
            if laid_link:
                add_link(p, L_link)  # Add link if randomly chosen
            p.run(dt)  # Move forward (maybe)
            p.tumble()  # Change direction (maybe)

    # Placeholder for optional post-processing (like merging links)
    #edges[:] = merge_collinear_links(edges)

def plot_network(nodes, edges, step=None):
    plt.figure(figsize=(6, 6))
    for start, end in edges:
        if np.linalg.norm(np.array(start) - np.array(end)) > 0.5:
            continue  # Skip links that wrap around the periodic boundary
        x_vals = [start[0], end[0]]
        y_vals = [start[1], end[1]]
        plt.plot(x_vals, y_vals, 'b-', alpha=0.5)  # Draw edge as blue line

    if nodes:
        xs, ys = zip(*nodes)
        plt.scatter(xs, ys, color='red', s=10, label='Nodes')  # Draw nodes as red dots

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().set_aspect('equal')  # Keep square aspect ratio
    plt.title(f"Network at Step {step}" if step is not None else "Network")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [13]:
def build_binned_network(edges, L_link_mean, tol=1e-5):
    box_size = L_link_mean
    M = int(np.ceil(1.0 / box_size))

    def get_box_indices(line):
        minx, miny, maxx, maxy = line.bounds
        ix_min, iy_min = int(minx // box_size), int(miny // box_size)
        ix_max, iy_max = int(maxx // box_size), int(maxy // box_size)
        return [(ix, iy)
                for ix in range(ix_min, ix_max + 1)
                for iy in range(iy_min, iy_max + 1)]

    def round_point(pt):
        return (round(pt.x, 5), round(pt.y, 5))

    # Step 1: Convert edge list into index-labeled LineStrings
    lines = []
    line_index = {}  # index → LineString
    grid = defaultdict(list)  # (ix, iy) → list of line indices

    for i, (start, end) in enumerate(edges):
        line = LineString([tuple(start), tuple(end)])
        lines.append(line)
        line_index[i] = line
        for box_idx in get_box_indices(line):
            grid[box_idx].append(i)
            
    # Step 2: Detect intersections per line with containment check
    intersections = defaultdict(set)  # line index → set of intersection points

    for ix in range(M):
        for iy in range(M):
            local_ids = []
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    neighbor = (ix + dx, iy + dy)
                    if neighbor in grid:
                        local_ids.extend(grid[neighbor])

            local_ids = list(set(local_ids))
            for i in range(len(local_ids)):
                for j in range(i + 1, len(local_ids)):
                    id1, id2 = local_ids[i], local_ids[j]
                    line1, line2 = line_index[id1], line_index[id2]
                    inter = line1.intersection(line2)
                    if isinstance(inter, Point):
                        pt = round_point(inter)
                        # Only add pt if it truly lies on the line segment
                        if line1.buffer(tol).contains(inter):
                            intersections[id1].add(pt)
                        if line2.buffer(tol).contains(inter):
                            intersections[id2].add(pt)

    # Step 3: Build graph from valid per-line intersections
    G = nx.Graph()

    for pts in intersections.values():
        for pt in pts:
            G.add_node(pt)

    for idx, pts in intersections.items():
        line = line_index[idx]
        pts_filtered = []
        for pt in pts:
            proj = line.project(Point(pt))
            if 0 <= proj <= line.length:
                pts_filtered.append(pt)

        if len(pts_filtered) < 2:
            continue

        pts_filtered.sort(key=lambda pt: line.project(Point(pt)))
        for i in range(len(pts_filtered) - 1):
            p1, p2 = pts_filtered[i], pts_filtered[i + 1]
            dist = np.linalg.norm(np.array(p1) - np.array(p2))
            G.add_edge(p1, p2, weight=dist)

    return G

def plot_graph(G):
    pos = {node: node for node in G.nodes()}  # Position = coordinate
    nx.draw(G, pos, node_size=30, node_color='red', edge_color='blue', with_labels=False)
    plt.gca().set_aspect('equal')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [14]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 100
N = 10
dt = 1.0

simulate(N=N, steps=steps, dt=dt, particle_params=params)

# 3. Optional: Visualize the raw links and deposition points
plot_network(nodes, edges, step=steps)

# 4. Build the network via binned spatial intersection detection
G = build_binned_network(edges, L_link_mean=params['L_link_mean'])

# 5. Visualize the final network graph
plot_graph(G)

In [21]:
# ── Ground‐truth test suites ─────────────────────────────────────────────────

# Each is a list of ((x1,y1),(x2,y2)) segments
ground_truths = {
    "X_cross": [ # Two diagonals crossing at (0.5, 0.5). Node at (0.5, 0.5), no edges
        ((0.2, 0.2), (0.8, 0.8)),
        ((0.2, 0.8), (0.8, 0.2))
    ],
    "T_cross": [ # T_cross: A horizontal and two vertical segments meeting in a T at (0.5, 0.5). Node at (0.5, 0.5), no edges
        ((0.1, 0.5), (0.9, 0.5)),
        ((0.5, 0.5), (0.5, 0.9)),
        ((0.5, 0.5), (0.5, 0.1))
    ],
    "parallel": [ # Two parallel lines. No nodes or intersections.
        ((0.1, 0.2), (0.9, 0.2)),
        ((0.1, 0.4), (0.9, 0.4))
    ],
    "triangle": [
        ((0.2, 0.2), (0.5, 0.8)), 
         ((0.8, 0.2), (0.2, 0.2)), 
         ((0.8, 0.2), (0.5, 0.8))
    ],
    "square": [
        ((0.2, 0.2), (0.2, 0.8)), 
        ((0.2, 0.8), (0.8, 0.8)), 
        ((0.8, 0.2), (0.2, 0.2)), 
        ((0.8, 0.2), (0.8, 0.8))
    ],
    "grid": [
        ((0.3, 0.3), (0.3, 0.7)), 
        ((0.3, 0.3), (0.7, 0.3)), 
        ((0.3, 0.7), (0.7, 0.7)), 
        ((0.7, 0.3), (0.7, 0.7))
    ],
    "chain": [((0.7, 0.5), (0.3, 0.5))],
}

def run_ground_truth_tests(builder_fn, L_link_mean=0.1):
    """
    builder_fn: function(edges, ...) → networkx.Graph
    """
    for name, edges in ground_truths.items():
        print(f"\n--- Test case: {name} ---")
        G = builder_fn(edges, L_link_mean=L_link_mean)
        print(" Nodes:", sorted(G.nodes()))
        print(" Edges:", sorted(G.edges()))
        plot_graph(G)  # if you want to see the layout

if __name__ == "__main__":
    # First, sanity‐check on a few tiny cases:
    run_ground_truth_tests(build_binned_network, L_link_mean=params['L_link_mean'])
    # Or, to test the full binned builder:
    # run_ground_truth_tests(build_binned_network, L_link_mean=params['L_link_mean'])
    
    # Now the real simulation:
    simulate(N=N, steps=steps, dt=dt, particle_params=params)
    plot_network(nodes, edges, step=steps)
    G = build_binned_network(edges, L_link_mean=params['L_link_mean'])
    plot_graph(G)


--- Test case: X_cross ---
 Nodes: [(0.5, 0.5)]
 Edges: []

--- Test case: T_cross ---
 Nodes: [(0.5, 0.5)]
 Edges: []

--- Test case: parallel ---
 Nodes: []
 Edges: []

--- Test case: triangle ---
 Nodes: [(0.2, 0.2), (0.5, 0.8), (0.8, 0.2)]
 Edges: [((0.2, 0.2), (0.8, 0.2)), ((0.5, 0.8), (0.2, 0.2)), ((0.5, 0.8), (0.8, 0.2))]

--- Test case: square ---
 Nodes: [(0.2, 0.2), (0.2, 0.8), (0.8, 0.2), (0.8, 0.8)]
 Edges: [((0.2, 0.2), (0.8, 0.2)), ((0.2, 0.8), (0.2, 0.2)), ((0.2, 0.8), (0.8, 0.8)), ((0.8, 0.2), (0.8, 0.8))]

--- Test case: grid ---
 Nodes: [(0.3, 0.3), (0.3, 0.7), (0.7, 0.3), (0.7, 0.7)]
 Edges: [((0.3, 0.3), (0.3, 0.7)), ((0.3, 0.3), (0.7, 0.3)), ((0.3, 0.7), (0.7, 0.7)), ((0.7, 0.3), (0.7, 0.7))]

--- Test case: chain ---
 Nodes: []
 Edges: []


In [22]:
def build_binned_network_debug(edges, L_link_mean, tol=1e-6):
    from shapely.geometry import LineString, Point
    from collections import defaultdict
    import networkx as nx
    import numpy as np

    print("🔨 Starting build_binned_network_debug")
    box_size = L_link_mean
    M = int(np.ceil(1.0/box_size))
    print(f"  box_size = {box_size:.3f}, grid M×M = {M}×{M}")

    def get_box_indices(line):
        minx, miny, maxx, maxy = line.bounds
        ix_min, iy_min = int(minx//box_size), int(miny//box_size)
        ix_max, iy_max = int(maxx//box_size), int(maxy//box_size)
        return [(ix,iy) 
                for ix in range(ix_min, ix_max+1) 
                for iy in range(iy_min, iy_max+1)]

    def round_pt(pt):
        return (round(pt.x,5), round(pt.y,5))

    # 1) Build the grid
    line_index = {}
    grid = defaultdict(list)
    print("\n1) Assigning links to boxes:")
    for i,(s,e) in enumerate(edges):
        line = LineString([s,e])
        line_index[i] = line
        boxes = get_box_indices(line)
        print(f"  link {i}: {s}→{e}, bounds {line.bounds}, boxes {boxes}")
        for b in boxes:
            grid[b].append(i)

    # 2) Find intersections
    intersections = defaultdict(set)
    print("\n2) Finding intersections in each box and neighbors:")
    for ix in range(M):
        for iy in range(M):
            # gather all lines in this cell + neighbors
            local = set()
            for dx in (-1,0,1):
                for dy in (-1,0,1):
                    local |= set(grid.get((ix+dx, iy+dy), []))
            if not local: 
                continue
            print(f"  cell ({ix},{iy}) has line IDs {sorted(local)}")
            L = list(local)
            for a in range(len(L)):
                for b in range(a+1, len(L)):
                    id1,id2 = L[a], L[b]
                    l1,l2 = line_index[id1], line_index[id2]
                    inter = l1.intersection(l2)
                    if inter.is_empty:
                        continue
                    print(f"    → Testing intersection link {id1} vs {id2}: {inter}")
                    # handle points and colinear overlaps
                    pts = []
                    if isinstance(inter, Point):
                        pts = [inter]
                    elif inter.geom_type=="LineString":
                        coords = list(inter.coords)
                        pts = [Point(coords[0]), Point(coords[-1])]
                        print(f"      colinear overlap, endpoints: {coords[0]}, {coords[-1]}")
                    else:
                        print("      skipping geom type", inter.geom_type)
                    for ip in pts:
                        rpt = round_pt(ip)
                        if l1.buffer(tol).contains(ip):
                            intersections[id1].add(rpt)
                        if l2.buffer(tol).contains(ip):
                            intersections[id2].add(rpt)

    # 3) Show raw intersection sets
    print("\n3) Raw intersections by link:")
    for idx, pts in intersections.items():
        print(f"  link {idx} got points {pts}")

    # 4) Build the graph
    G = nx.Graph()
    print("\n4) Building graph edges:")
    for idx, pts in intersections.items():
        line = line_index[idx]
        valid = [pt for pt in pts if 0<=line.project(Point(pt))<=line.length]
        valid.sort(key=lambda pt: line.project(Point(pt)))
        print(f"  link {idx}, sorted pts on segment: {valid}")
        for u,v in zip(valid, valid[1:]):
            d = np.linalg.norm(np.array(u)-np.array(v))
            print(f"    → adding edge {u}–{v}, dist {d:.3f}")
            G.add_edge(u, v, weight=d)

    print("\n✔️ Done.")
    return G

In [27]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 10
N = 10
dt = 1.0

simulate(N=N, steps=steps, dt=dt, particle_params=params)
plot_network(nodes, edges, step=steps)
G = build_binned_network_debug(edges, L_link_mean=0.1)
plot_graph(G)

🔨 Starting build_binned_network_debug
  box_size = 0.100, grid M×M = 10×10

1) Assigning links to boxes:
  link 0: [0.44835414 0.84466592]→[0.31664922 0.83691492], bounds (0.3166492179506579, 0.8369149170701247, 0.4483541422817905, 0.8446659168241248), boxes [(3, 8), (4, 8)]
  link 1: [0.03333442 0.00595433]→[0.99696087 0.89909962], bounds (0.03333441590084722, 0.005954332029906695, 0.9969608674395244, 0.899099619101099), boxes [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7),